In [408]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys
import bs4
import pprint
import pandas as pd
pd.options.display.max_rows = 1000

import cufflinks as cf
from plotly.offline import iplot
import plotly.graph_objects as go
import plotly.offline as py

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("profile")
options.add_argument("user-data-dir=./webtelegram")

In [6]:
driver = webdriver.Chrome(executable_path = "./chromedriver", options=options)
driver.get('https://web.telegram.org/#/im?p=c1474749462_9934961634770853506')

In [128]:
htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
messages = htmlPage.find_all('div', {'class':'im_history_message_wrap'})

In [11]:
exchanges = ['BET365', 'PINNACLE', '188BET', 'DAFABET', '10BET', 'BETFAIR', 'BETWAY', '10BET']

In [324]:
opportunities = []
for message in messages:
    bkp = list(exchanges)
    primeira, segunda = False, False
    
    for ex1 in exchanges:
        if ex1 in message.text:
            primeira = True
            bkp.remove(ex1)
            break
    
    for ex2 in bkp:
        if ex2 in message.text:
            segunda = True
            break

    if primeira and segunda:
        opportunities.append(message)
#         message = message
#         print(ex1, ex2)

'10:26 AM'

In [352]:
op = []
for message in opportunities:
    hour = message.find('span', {'ng-bind':'::historyMessage.date | time'}).text
    
    message = message.text.replace('\n', '')
    #parametros para separar os dados importantes
    p1 = message.find('9)')
    p2 = message[p1+1:].find('(') + p1
    first_exchange = message[p1:p2]
    p3 = message[p2+5:].find('(') + p2
    second_exchange = message[p2:p3]

    #para pegar os links
    if 'DAFABET' not in message:
        http1 = message.find('http')
        http2 = message[http1+5:].find('http') + http1
        link1 = message[http1:http2+5]
        link2 = message[http2+5:]
    else:
        http1 = message.find('http')
        ref1 = message[http1:].find(' p/ Desktop') + http1 + 11 #+11 para nao incluir o texto desktop na escrita
        link1 = message[http1:ref1]
        http2 = message[ref1:].find(' ') + ref1
        link2 = message[ref1:http2]
        
    data = {'time': hour,'ex1': first_exchange, 'ex2': second_exchange, 'link1': link1, 'link2': link2}
    op.append(data)

In [354]:
df = pd.DataFrame(op)

In [376]:
for row, content in enumerate(df['time']):
    timee = content.split(':')
    
    if timee[-1][-2:] == 'PM':
        timee = int(timee[0]) + 12
    else:
        timee = int(timee[0])
    df.loc[row, 'hour'] = timee

In [394]:
hours = df['hour'].value_counts().sort_index()

In [412]:
dados = [go.Bar(x=hours.index, y=hours.values)]

configuracoes_layout =  go.Layout(title='Horarios que mais ocorrem oportunidades', yaxis={'title':'Quantidade'}, xaxis={'title': 'Hora do Dia'})

figura = go.Figure(data=dados, layout=configuracoes_layout)

# figura.write_image('oportunidades.png')
py.iplot(figura)

In [413]:
messages_driver = driver.find_elements_by_class_name('im_history_message_wrap')

In [ ]:
from twilio.rest import Client

account_sid = 'AC9c0f990c8862e7233314e7cb4c763078'
auth_token = '9e69cd4edea02c36f50f33483bbbdd3d'
client = Client(account_sid, auth_token)

# this is the Twilio sandbox testing number
from_whatsapp_number='whatsapp:+14155238886'
# replace this number with your own WhatsApp Messaging number
yan_number='whatsapp:+554788917321'
uilton_number='whatsapp:+557191289667'

def sendMessage(message):
#     client.messages.create(body=message, from_=from_whatsapp_number, to=yan_number)
    client.messages.create(body=message, from_=from_whatsapp_number, to=uilton_number)
    return True


sendMessage('Teste', )

In [ ]:
def createMessage(message):
    '''
        Recebe o conteudo e cria uma mensagem para ser mandada no whatsapp
    '''
    
    content = message.splitlines()
#     return content
    ex_a = False
    link_a = False
    
    try:
        for pos, line in enumerate(content):
            if 'STAKE' in line and ex_a == False:
                ex_a = line
            elif 'STAKE' in line and ex_a != False:
                ex_b = line

            elif 'LUCRO' in line:
                lucro = line

            elif 'https:' in line and link_a == False:
                link_a = line
            elif 'https:' in line and link_a != False:
                link_b = line

        message = ex_a + '\n\n' + ex_b + '\n\n' +  lucro + '\n\n' + link_a + '\n\n' + link_b
    except:
        pass
    return message

In [160]:
''' IRA TRATAR O CONTEUDO DA PRIMEIRA LINHA'''

#divisor de porcentagem e odd
divisor = lines[0].find('/')

#encontra a porcentagem e pega os dois caracteres numericos que estao antes
percent = lines[0][:divisor].find('%')

#porcentagem que ira na exchange 1
percent_1 = int(lines[0][:divisor][percent-2:percent])

#primeira exchange
divisor_exchange_1 = lines[0].find('-')
exchange_1 = lines[0][:divisor_exchange_1].replace('.','')

# pega os separadores
divisor_D = lines[0][divisor:].find('DD')
divisor_parentesis = lines[0][divisor:].find('(')

#transforma em float o conteudo que esta entre ~ODD~ e o simbolo de parentesis, ou seja, a ODD do evento
odd = float(lines[0][divisor:][divisor_D+2:divisor_parentesis].replace(' ','').replace(',','.'))

In [ ]:
# BETWAY = 163
# 188BET = 72
# PINNACLE 68
# BET365 = 196
# MARATHONBET = 204
# VBET = 59